In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Sequence
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm.exc import NoResultFound
from sqlalchemy.types import ARRAY

Base = declarative_base()

class User(Base):
    __tablename__ = 'user'
    id = Column(Integer, Sequence('user_id_seq'), primary_key=True)
    firstname = Column(String, nullable = True)
    lastname = Column(String, nullable = True)
    answers = Column(ARRAY(String), nullable = True)
    def __repr__(self):
        return 'id: %d, name: %s %s, answers: %s'%(self.id, self.firstname, self.lastname, self.answers)

class DataRemote:
    __engine = None

    def __init__(self):
        self.__engine = create_engine('sqlite:///data.db', echo=False)
    def add_user(self, user):
        Base.metadata.create_all(self.__engine)
        Session = sessionmaker(bind=self.__engine)
        session = Session()
        u = User(
            firstname = user['firstname'],
            lastname = user['lastname'],
            requests = user['answers']
        )
        session.add(u)
        session.commit()
        session.close()
        return {"user" : "added!!"}
    def get_users(self):
        Session = sessionmaker(bind=self.__engine)
        session = Session()
        all_users =  session.query(User).all()
        list_users = []
        for u in all_users:
            u_json = {
                "id" : u.id,
                "firstname" : u.firstname,
                "lastname" : u.lastname,
                "answers" : u.answers 
            }
            list_users.append(u_json)
        session.close()
        return list_users
    def get_user_by_id(self,user_id):
        Session = sessionmaker(bind=self.__engine)
        session = Session()
        try:
            u =  session.query(User).filter(User.id==user_id).one()
            u_json = {
                "id" : u.id,
                "firstname" : u.firstname,
                "lastname" : u.lastname,
                "answers" : u.answers 
            }
        except NoResultFound as e:
            return {"Dont have id" : user_id}
        finally:
            session.close()
        return u_json
    def delete_user(self, user_id):
        Session = sessionmaker(bind=self.__engine)
        session = Session()
        try: 
            tryget = session.query(User).filter(User.id==user_id).one()
            session.delete(tryget)
            session.commit()
            return {
                "Deleted!" : user_id
            }
        except:
            return {
                "No User ID" : user_id
            }
        finally: 
            session.close()
    def update_user(self, user):
        Session = sessionmaker(bind=self.__engine)
        session = Session()
        try:
            a = session.query(User).filter(User.id==user['id']).one()
            a.fullname = user['firstname']
            a.nickname = user['lastname']
            a.answers = user['answers']
            session.commit()
            return {
                "updated User id" : user['id']
            }
        except:
            return {
                "cannot update User id" : user['id']
            }
        finally:
            session.close()



In [2]:
u1 = User()
remote = DataRemote()

In [3]:
remote

In [4]:
remote.add_user(u1)

CompileError: (in table 'user', column 'answers'): Compiler <sqlalchemy.dialects.sqlite.base.SQLiteTypeCompiler object at 0x000001EFD4CE9BC8> can't render element of type <class 'sqlalchemy.sql.sqltypes.ARRAY'>